In [2]:
# from django.db import models
from account.models import User
import datetime as dt
import json
import os
# from django.utils.translation import gettext_lazy as _
# Create your models here.

def house_image_main(instance, filename):
	upload_to = 'home_images'
	ext = filename.split('.')[-1]
	# get filename
	filename = '{}/home_{}_main.{}'.format(instance.id, instance.id, ext)
	# return the whole path to the file
	return os.path.join(upload_to, filename)

class House(models.Model):
	id = models.IntegerField(primary_key=True)
	user = models.OneToOneField(User, on_delete = models.SET_NULL, null=True, blank = True)
	street_address = models.CharField(max_length = 40)
	city = models.CharField(max_length = 24, null=True)
	state = models.CharField(max_length = 2, null=True,
		help_text = "Two character representation of the state wherein this house resides.")
	zip_code = models.CharField(max_length = 5)
	price = models.IntegerField()
	beds = models.PositiveSmallIntegerField()
	baths_2x = models.PositiveSmallIntegerField()
	square_footage = models.IntegerField(null=True)
	lot_size = models.IntegerField(null=True)
	year_built = models.PositiveSmallIntegerField(null=True)
	monthly_hoa = models.IntegerField(null=True)
	latitude = models.FloatField(null=True)
	longitude = models.FloatField(null=True)
	redfin_url = models.URLField()
	property_type = models.CharField(max_length = 50, null=True)
	location = models.CharField(max_length = 50, null=True)
	dollars_per_sqfeet = models.IntegerField(null=True)
	next_open_house_start_time = models.DateTimeField(null=True)
	next_open_house_end_time = models.DateTimeField(null=True)
	impressions = models.IntegerField(default=0)
	days_on_market = models.IntegerField(null=True)
	elevation = models.PositiveIntegerField(null=True)
	freeway_distance = models.FloatField(null=True)
	radiation_distance = models.FloatField(null=True)
	main_image = models.ImageField(null=True, upload_to=house_image_main)
	PENDING = 0
	ACTIVE = 1
	PRE_ON_MARKET = 2
	SOLD = 3
	CONTINGENT = 4
	status_choices = (
		(PENDING, "Pending"),
		(ACTIVE, "Active"),
		(PRE_ON_MARKET, "Pre On-Market"),
		(SOLD, "Sold"),
		(CONTINGENT, "Contingent")
	)
	status = models.PositiveSmallIntegerField(choices = status_choices, null=True)
	date_last_updated = models.DateField(_("Date"), default=dt.date.today, auto_now=False, auto_now_add=False)
	date_created = models.DateField(_("Date"), default=dt.date.today, auto_now=False, auto_now_add=False)
	sold_date = models.DateField(_("Date"), null=True, auto_now=False, auto_now_add=False)
	agent_info = models.TextField(null=True)
	noise_rating = models.FloatField(null=True)
	safety_rating = models.FloatField(null=True)
	community_rating = models.FloatField(null=True)
	info = models.TextField(null=True)
	parking = models.TextField(null=True)
	interior = models.TextField(null=True)
	exterior = models.TextField(null=True)
	financial = models.TextField(null=True)
	utilities = models.TextField(null=True)


ModuleNotFoundError: ignored

In [ ]:
	def get_bathrooms(self) -> float:

		return self.baths_2x/2 if self.baths_2x else None

	def get_address(self) -> str:
		'''
		Returns the address in form "street_address, city, state zip_code".
		'''

		return (self.street_address + ", " + self.city + ", " + self.state + " "
			+ self.zip_code)

	def get_num_comments(self) -> int:
		'''
		Returns the number of comments and replies for object.
		'''
		raise NotImplementedError()



	def get_json(self) -> dict:
		'''
		Returns a JSON represnetation (dict) of the current object.
		'''

		dictionary = {
		"street_address": self.street_address,
		"city": self.city,
		"state": self.state,
		"zip_code": self.zip_code,
		"info": self.info,
		"id": self.id,
		"latitude": self.latitude,
		"longitude": self.longitude,
		"main_image": self.main_image.url if self.main_image else "",
		}

		if self.user != None:
			dictionary["user_id"] = self.user.id

		return dictionary


class School(models.Model):
	houses = models.ManyToManyField(House)
	id = models.PositiveIntegerField(primary_key=True)
	virltor_rating = models.FloatField(null=True)
	name = models.CharField(max_length=255)
	private = models.BooleanField(null=True)
	grades = models.CharField(null=True, max_length=255)
	student_body_size = models.PositiveIntegerField(null=True)
	students_per_teacher = models.SmallIntegerField(null=True)
	address = models.CharField(max_length=255)
	school_district = models.CharField(null=True, max_length=255)
	community_rating = models.FloatField(null=True)
	total_community_reviews = models.SmallIntegerField(null=True)
	great_schools_rating = models.SmallIntegerField(null=True)
	url = models.CharField(max_length=255)
	date_last_updated = models.DateField(_("Date"), default=dt.date.today, auto_now=False, auto_now_add=False)
	latitude = models.FloatField(null=True)
	longitude = models.FloatField(null=True)


class SchoolRating(models.Model):
	school = models.ForeignKey(School, on_delete=models.CASCADE)
	text = models.CharField(null=True, max_length=500)
	sentiment_rating = models.FloatField(null=True)


class CrimeProfile(models.Model):
	'''A CrimeProfile is the table which holds which crimes relate to
		which house'''

	house = models.OneToOneField(House, on_delete = models.CASCADE, null=True)

	# Crimes are stored in one big string, back to back.
	crimes = models.TextField(default = "")

	crime_score = models.FloatField(null=True)

	crime_ranking = models.IntegerField(null=True)

	# The length of each crime string
	CRIME_LENGTH = 22
	def add_crime(self, crime):
		'''Appends a crime to the CrimeProfile; does not save to the database.
			"crime" must have fields crime: int, latitude: float, longitude: float,
			time: datetime'''

		# "c" is the crime id, "a" the latitude, and "o" the longitude.
		# The format for storing each crime, as a string, is
		# ccaaaaaaaaaooooooooo
		# This assumes that no crime has an id with more than two digits.
		crime_id = "{:02d}".format(crime.crime)
		lat = "{:010.5f}".format(crime.latitude)
		lon = "{:010.5f}".format(crime.longitude)

		self.crimes += crime_id + lat + lon

	def get_crime_tuples(self) -> list:
		'''Returns all crimes pretaining to the profile in the form of a list.
			The list contains tuples with three entrees formatted like so:
			(crime_id: int, latitude: float, longitude: float)'''

		CL = self.CRIME_LENGTH
		num_crimes = len(self.crimes) // CL

		crime_tuples = [(1, 1.1, 1.1)] * num_crimes

		for i in range(num_crimes):
			crime_tuples[i] = (int(self.crimes[i*CL: i*CL + 2]), float(self.crimes[i*CL+ 2: i*CL + 2 + (CL - 2)//2]),
				float(self.crimes[i*CL + 2 + (CL - 2)//2:(i + 1)*CL]))

		return crime_tuples


class Comment(models.Model):

	house = models.ForeignKey(House, on_delete = models.CASCADE)
	parent_comment = models.ForeignKey("self", on_delete = models.CASCADE, related_name = 'replies', null=True)

	user = models.ForeignKey(User, null=True, on_delete = models.SET_NULL)

	creation_timestamp = models.DateTimeField(auto_now_add = True)

	update_timestamp = models.DateTimeField(auto_now = True)

	body = models.TextField(max_length = 2000)


	class Meta():
		ordering = ('-creation_timestamp',)

	def is_reply(self):
		return self.parent_comment != None

	def get_replies(self):
		'''Returns all direct replies to this comment'''

		return Comment.objects.filter(parent_comment = self)

	def get_num_replies_recrusive(self):
		'''Returns the number of replies to this comment. Recursively counts replies of replies of replies... '''
		replies = Comment.objects.filter(parent_comment = self)

		number = 0
		for reply in replies:
			number += 1
			number += reply.get_num_replies_recursive()

		return number

	def like(self, user):
		'''Likes comment on behalf of "user" '''

		s = Sentiment.objects.filter(user = user, comment = self)

		if len(s) == 0:
			Sentiment(user = user, comment = self, sentiment = True).save()
			return

		s, = s
		s.sentiment = True

		s.save()


	def dislike(self, user):
		'''Dislikes comment on behalf of "user" '''
		s = Sentiment.objects.filter(user = user, comment = self)

		if len(s) == 0:
			Sentiment(user = user, comment = self, sentiment = False).save()
			return

		s, = s

		s.sentiment = False

		s.save()

	def unrate(self, user):
		'''Unrates comment on behalf of "user" '''
		Sentiment.objects.get(user = user, comment = self).delete()

	def get_ratings(self) -> tuple:
		''' Returns a touble, (likes, dislikes), where likes is the number
				of likes the post has and dislikes is the number of dislikes.'''
		likes = 0
		dislikes = 0
		for sentiment in self.sentiment_set.all():

			if sentiment.sentiment == True:
				likes += 1
			else:
				dislikes += 1

		return (likes, dislikes)

	def get_user_sentiment(self, user) -> int:
		'''Returns 1 if "user" has liked this comment, -1 if "user" has disliked it,
			and 0 if "user" has niether liked nor disliked it.'''

		sentiment = Sentiment.objects.filter(comment = self, user = user).first()

		if sentiment == None:
			return 0
		elif sentiment.sentiment == False:
			return -1

		# sentiment.sentiment == True
		return 1


	def get_json(self, user = None) -> dict:
		'''Returns a JSON represnetation (dict) of the current object.

		:param user: If user does not equal None, an extra key is added
			to the dictionary, "sentiment" which has value 1 if user has liked the comment,
			0 if the user has neither liked nor disliked the comment, and -1 if the user
			has disliked the comment.
		'''
		likes, dislikes = self.get_ratings()

		dictionary = {
		"comment_id": self.pk,
		"username": self.user.username,
		"house_id": self.house.pk,
		"creation_timestamp": self.creation_timestamp.isoformat(),
		"update_timestamp": self.update_timestamp.isoformat(),
		"body": self.body,
		"like_count": likes,
		"dislike_count": dislikes,
		}

		if user != None:
			dictionary["sentiment"] = self.get_user_sentiment(user)
			dictionary["user_id"] = self.user.id

		return dictionary


class Sentiment(models.Model):
	user = models.ForeignKey(User, on_delete = models.CASCADE)
	comment = models.ForeignKey(Comment, on_delete = models.CASCADE)

	# True means 'like' and False means 'dislike'
	sentiment = models.BooleanField()

	class Meta():
		unique_together = ['user', 'comment']